In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np


%matplotlib inline

找到資料夾內所有的檔案

In [2]:
import os

path = ['.\images\M','.\images\J','.\images\\N','.\images\S','.\images\Z']
files = [[],[],[],[],[]]
# r=root, d=directories, f = files
for index in range(5):
    for r, d, f in os.walk(path[index]):
        for file in f:
            files[index].append(os.path.join(r, file))
'''
for i in range(5):
    for f in files[i]:
        print(f)
    print('----')
'''

"\nfor i in range(5):\n    for f in files[i]:\n        print(f)\n    print('----')\n"

In [3]:
import face_recognition
import cv2

image with problem: 

'.\images\M\敏珠10.jpg'

'.\images\J\jessica4.jpg',

'.\images\J\jessica10.jpg',

In [4]:

# 最後要記錄出的樣品向量(128 維度) 和標籤(明星名)
encodinglist = []
labels = []
# 一橫列畫幾個人
width = 5

for index_of_person in range(5):
    # 總共幾個橫列
    height = int(len(files[index_of_person]) / width) + 1
    # 整個大圖的size
    plt.figure(figsize=(100,100))
    #print('------------')
    for (i, f) in enumerate(files[index_of_person]):
        #if i >540: 
        #if f == (".\images\S\SinB.full.160333.jpg"): 
        # Step1. 讀取檔案
            img = face_recognition.load_image_file(f)
        # Step2. 把臉的降維向量算出, 用已經做好的cnn, [0] 第一張臉
            face_detect_and_encoding = face_recognition.face_encodings(img)
            #print(face_encoding[0])
        # Step3. 抓出臉的方框, 我們這裡沒用到, 但如果你想畫框就會需要
        # 這裡的face_locations 可以使用cnn(深度學習) 或者haar(固定特徵) 來得到
        # 如果需要快速, 請使用haar, 如果需要更準確, 就使用cnn, 建議可以使用預設haar 即可
            if len(face_detect_and_encoding) <= 0:
                print("No faces found in the image!")
                continue
            else :
                face_encoding = face_detect_and_encoding[0]

            print(f+", NO:"+str(i))

            height,width = img.shape[:2] #get image height and width
            if(height>1500 or width>1500):
                img = cv2.resize(img,(width//3,height//3),interpolation=cv2.INTER_CUBIC)
            elif(height>800 or width>800):
                img = cv2.resize(img,(width//2,height//2),interpolation=cv2.INTER_CUBIC)
            (top, right, bottom, left) = face_recognition.face_locations(img, model="cnn")[0]

            #draw
            #size = int(img.shape[0] / 100)
            #cv2.rectangle(img, (left, top), (right, bottom), (100, 100, 200),thickness=3)
            #print(str(right) + ' '+str(top) + ' '+str(bottom) + ' '+str(left))
        # Step4. 把它加到我自己準備的list 裡
            encodinglist.append(face_encoding)
        # Step6. 把人名到我自己準備的list
            labels.append(index_of_person)
        # 利用enumerate 得到的i 指定subplot
            #plt.subplot(height, width, i + 1)
            #plt.axis("off")
            #plt.imshow(img)

.\images\M\22 (1).jpeg, NO:0
.\images\M\22 (10).jpeg, NO:1
.\images\M\22 (2).jpeg, NO:2
.\images\M\22 (3).jpeg, NO:3
.\images\M\22 (4).jpeg, NO:4
.\images\M\22 (5).jpeg, NO:5
.\images\M\22 (6).jpeg, NO:6
.\images\M\22 (7).jpeg, NO:7
.\images\M\22 (8).jpeg, NO:8
.\images\M\22 (9).jpeg, NO:9
.\images\M\4.jpg, NO:10
.\images\M\42908264_125217361795258_4649541283885433978_n.jpeg, NO:11
.\images\M\43720896_2154500968211344_1165963856466626970_n.jpeg, NO:12
.\images\M\43820733_2004915812920676_5911789304081739526_n.jpeg, NO:13
.\images\M\43914530_687151141685328_266911156013580554_n.jpeg, NO:14
.\images\M\44453841_140055326956315_8254950549910955970_n.jpeg, NO:15
.\images\M\49487633_807408806320204_6993398693486091578_n.jpeg, NO:16
.\images\M\50000887_2118363665143597_9039670849918355634_n.jpeg, NO:17
.\images\M\50254792_2621606434532592_1698376544037944433_n.jpeg, NO:18
.\images\M\50520800_346340142642552_2298609155863018384_n.jpeg, NO:19
.\images\M\50924448_1188578084640612_188122642004259

.\images\M\M (1).png, NO:194
.\images\M\M (10).jpg, NO:195
No faces found in the image!
.\images\M\M (100).png, NO:197
.\images\M\M (101).png, NO:198
.\images\M\M (102).png, NO:199
.\images\M\M (103).png, NO:200
.\images\M\M (104).png, NO:201
.\images\M\M (105).png, NO:202
.\images\M\M (106).png, NO:203
.\images\M\M (107).png, NO:204
.\images\M\M (108).png, NO:205
.\images\M\M (109).png, NO:206
.\images\M\M (11).jpg, NO:207
.\images\M\M (11).png, NO:208
.\images\M\M (110).png, NO:209
.\images\M\M (111).png, NO:210
.\images\M\M (112).png, NO:211
.\images\M\M (113).png, NO:212
.\images\M\M (114).png, NO:213
.\images\M\M (115).png, NO:214
.\images\M\M (116).png, NO:215
.\images\M\M (117).png, NO:216
.\images\M\M (118).png, NO:217
.\images\M\M (119).png, NO:218
.\images\M\M (12).jpg, NO:219
.\images\M\M (12).png, NO:220
.\images\M\M (120).png, NO:221
.\images\M\M (121).png, NO:222
.\images\M\M (122).png, NO:223
.\images\M\M (123).png, NO:224
.\images\M\M (124).png, NO:225
.\images\M\M (125

.\images\M\M (39).png, NO:461
.\images\M\M (4).jpg, NO:462
.\images\M\M (4).png, NO:463
.\images\M\M (40).png, NO:464
.\images\M\M (41).png, NO:465
.\images\M\M (42).png, NO:466
.\images\M\M (43).png, NO:467
.\images\M\M (44).png, NO:468
.\images\M\M (45).png, NO:469
.\images\M\M (46).png, NO:470
.\images\M\M (47).png, NO:471
.\images\M\M (48).png, NO:472
.\images\M\M (49).png, NO:473
.\images\M\M (5).jpg, NO:474
.\images\M\M (5).png, NO:475
.\images\M\M (50).png, NO:476
.\images\M\M (51).png, NO:477
.\images\M\M (52).png, NO:478
.\images\M\M (53).png, NO:479
.\images\M\M (54).png, NO:480
.\images\M\M (55).png, NO:481
.\images\M\M (56).png, NO:482
.\images\M\M (57).png, NO:483
.\images\M\M (58).png, NO:484
.\images\M\M (59).png, NO:485
.\images\M\M (6).jpg, NO:486
.\images\M\M (6).png, NO:487
.\images\M\M (60).png, NO:488
.\images\M\M (61).png, NO:489
.\images\M\M (62).png, NO:490
.\images\M\M (63).png, NO:491
.\images\M\M (64).png, NO:492
.\images\M\M (65).png, NO:493
.\images\M\M (66

.\images\J\000 (39).jpeg, NO:89
.\images\J\000 (4).jpeg, NO:90
.\images\J\000 (40).jpeg, NO:91
.\images\J\000 (41).jpeg, NO:92
.\images\J\000 (42).jpeg, NO:93
.\images\J\000 (43).jpeg, NO:94
.\images\J\000 (44).jpeg, NO:95
.\images\J\000 (45).jpeg, NO:96
.\images\J\000 (46).jpeg, NO:97
.\images\J\000 (47).jpeg, NO:98
.\images\J\000 (48).jpeg, NO:99
.\images\J\000 (49).jpeg, NO:100
.\images\J\000 (5).jpeg, NO:101
.\images\J\000 (50).jpeg, NO:102
No faces found in the image!
.\images\J\000 (52).jpeg, NO:104
.\images\J\000 (53).jpeg, NO:105
.\images\J\000 (54).jpeg, NO:106
.\images\J\000 (55).jpeg, NO:107
.\images\J\000 (56).jpeg, NO:108
No faces found in the image!
No faces found in the image!
.\images\J\000 (59).jpeg, NO:111
.\images\J\000 (6).jpeg, NO:112
.\images\J\000 (60).jpeg, NO:113
.\images\J\000 (61).jpeg, NO:114
.\images\J\000 (62).jpeg, NO:115
No faces found in the image!
.\images\J\000 (64).jpeg, NO:117
.\images\J\000 (65).jpeg, NO:118
No faces found in the image!
No faces fo

.\images\J\images (22).jpg, NO:306
.\images\J\images (24).jpg, NO:307
.\images\J\images (25).jpg, NO:308
.\images\J\images (26).jpg, NO:309
.\images\J\images (27).jpg, NO:310
.\images\J\images (29).jpg, NO:311
.\images\J\images (3).jpg, NO:312
.\images\J\images (30).jpg, NO:313
.\images\J\images (32).jpg, NO:314
.\images\J\images (33).jpg, NO:315
.\images\J\images (35).jpg, NO:316
.\images\J\images (39).jpg, NO:317
.\images\J\images (4).jpg, NO:318
.\images\J\images (40).jpg, NO:319
.\images\J\images (41).jpg, NO:320
.\images\J\images (42).jpg, NO:321
No faces found in the image!
.\images\J\images (44).jpg, NO:323
No faces found in the image!
.\images\J\images (51).jpg, NO:325
.\images\J\images (52).jpg, NO:326
.\images\J\images (53).jpg, NO:327
.\images\J\images (55).jpg, NO:328
.\images\J\images (57).jpg, NO:329
.\images\J\images (6).jpg, NO:330
.\images\J\images (60).jpg, NO:331
.\images\J\images (65).jpg, NO:332
.\images\J\images (7).jpg, NO:333
.\images\J\images (70).jpg, NO:334
N

...

<Figure size 7200x7200 with 0 Axes>

<Figure size 7200x7200 with 0 Axes>

<Figure size 7200x7200 with 0 Axes>

<Figure size 7200x7200 with 0 Axes>

<Figure size 7200x7200 with 0 Axes>

In [5]:
%env KERAS_BACKEND=tensorflow
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import Adam ,SGD

from sklearn.model_selection import train_test_split

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [6]:
# 秀一下降維過後的向量
print("x長度:", len(encodinglist))
print("y長度:", len(labels))

data_length = len(encodinglist)

data_x = np.array(encodinglist)
data_y = np.array(labels)

data_x = data_x.reshape(data_length, 128, 1, 1)

data_y = np_utils.to_categorical(data_y, 5)

x_train,x_test,y_train,y_test =train_test_split( data_x , data_y ,test_size = 0.15, random_state = 4)


print("x長度:", len(y_train))
print("y長度:", len(y_test))

x長度: 3112
y長度: 3112
x長度: 2645
y長度: 467


以上已經把training data 的 128維都已經存在array之中，接下來開始做CNN判斷

TODO: input normalize

In [7]:
model1 = Sequential()

model1.add(Conv2D(40, (4,1), padding='same', input_shape=(128,1,1))) #4個filter，都是5*5
model1.add(Activation('relu'))
model1.add(MaxPool2D(pool_size=(2,1)))

model1.add(Conv2D(60, (4,1), padding='same'))
model1.add(Activation('relu'))
model1.add(MaxPool2D(pool_size=(2,1)))

model1.add(Conv2D(80, (4,1), padding='same'))
model1.add(Activation('relu'))
model1.add(MaxPool2D(pool_size=(2,1)))

model1.add(Conv2D(100, (4,1), padding='same'))
model1.add(Activation('relu'))
model1.add(MaxPool2D(pool_size=(2,1)))

model1.add(Flatten())
model1.add(Dense(150))#拉平完送進最後一個普通NN
model1.add(Activation('relu'))

model1.add(Dense(5))
model1.add(Activation('softmax'))

model1.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model1.summary()

model1_out=model1.fit(x_train, y_train, batch_size=3, epochs=40,verbose =1,validation_data = (x_train,y_train))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 1, 20)        100       
_________________________________________________________________
activation_1 (Activation)    (None, 128, 1, 20)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 1, 20)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 1, 40)         3240      
_________________________________________________________________
activation_2 (Activation)    (None, 64, 1, 40)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 1, 40)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 1, 60)         9660      
__________

2. Normal NN

In [8]:
# 秀一下降維過後的向量
print("x長度:", len(encodinglist))
print("y長度:", len(labels))

data_length = len(encodinglist)

NN_inputx = np.array(encodinglist)
NN_inputy = np.array(labels)

NN_inputy = np_utils.to_categorical(NN_inputy, 5)

NN_x_train,NN_x_test,NN_y_train,NN_y_test =train_test_split( NN_inputx , NN_inputy ,test_size = 0.15, random_state = 4)


print("x長度:", len(y_train))
print("y長度:", len(y_test))

x長度: 3112
y長度: 3112
x長度: 2645
y長度: 467


In [9]:

model2 = Sequential()
model2.add(Dense(200, input_dim=128))
model2.add(Activation("relu"))
model2.add(Dense(100))
model2.add(Activation("relu"))
model2.add(Dense(5))
model2.add(Activation("softmax"))
model2.compile(loss='mse', optimizer='Adam', metrics=['accuracy'])
model2.summary()

model2_out = model2.fit(NN_x_train, NN_y_train,
                        batch_size = 3,
                        epochs = 40,
                        verbose = 1,
                        validation_data = (NN_x_test, NN_y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 200)               25800     
_________________________________________________________________
activation_7 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               20100     
_________________________________________________________________
activation_8 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 505       
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
Total params: 46,405
Trainable params: 46,405
Non-trainable params: 0
_________________________________________________________________
Train 

3. Euclidean Distance

In [10]:
# 秀一下降維過後的向量
print("x長度:", len(encodinglist))
print("y長度:", len(labels))

data_length = len(encodinglist)

E_inputx = np.array(encodinglist)
E_inputy = np.array(labels)
E_x_train,E_x_test,E_y_train,E_y_test =train_test_split( E_inputx , E_inputy ,test_size = 0.15, random_state = 4)


print("x長度:", len(y_train))
print("y長度:", len(y_test))

x長度: 3112
y長度: 3112
x長度: 2645
y長度: 467


In [11]:
mean0,mean1,mean2,mean3,mean4=[],[],[],[],[]
for i in range(len(E_y_train)):
    if ((E_y_train[i]==0)):
        if(len(mean0)==0):
            mean0.append(E_x_train[i])
        else:
            mean0+=E_x_train[i]
    elif (E_y_train[i]==1):
        if(len(mean1)==0):
            mean1.append(E_x_train[i])
        else:
            mean1+=E_x_train[i]
    elif (E_y_train[i]==2):
        if(len(mean2)==0):
            mean2.append(E_x_train[i])
        else:
            mean2+=E_x_train[i]
    elif (E_y_train[i]==3):
        if(len(mean3)==0):
            mean3.append(E_x_train[i])
        else:
            mean3+=E_x_train[i]
    elif (E_y_train[i]==4):
        if(len(mean4)==0):
            mean4.append(E_x_train[i])
        else:
            mean4+=E_x_train[i]
            
mean0/=len(encodinglist)
mean1/=len(encodinglist)
mean2/=len(encodinglist)
mean3/=len(encodinglist)
mean4/=len(encodinglist)

上面已經取出每個人的平均了

In [12]:
'''
print(np.mean(encodinglist, axis=0))
mean=0
for i in range(len(encodinglist)):
    mean=mean+encodinglist[i][0]
mean/len(encodinglist)
'''

'\nprint(np.mean(encodinglist, axis=0))\nmean=0\nfor i in range(len(encodinglist)):\n    mean=mean+encodinglist[i][0]\nmean/len(encodinglist)\n'

In [14]:
from scipy.spatial import distance

right=0
wrong=0
for i in range(len(E_y_test)):
    dst0 = distance.euclidean(mean0,E_x_test[i])
    dst1 = distance.euclidean(mean1,E_x_test[i])
    dst2 = distance.euclidean(mean2,E_x_test[i])
    dst3 = distance.euclidean(mean3,E_x_test[i])
    dst4 = distance.euclidean(mean4,E_x_test[i])
    dst = [dst0,dst1,dst2,dst3,dst4]
    predict = dst.index(min(dst))
    #print(str(dst0)+" , "+str(dst1)+" , "+str(dst2)+" , "+str(dst3)+" , "+str(dst4))
    #print("預測: "+str(predict)+" , 真實: ",str(E_y_test[i]))
    if(predict==E_y_test[i]):
        right+=1
    else:
        wrong+=1
print("Euclidean Distance 正確率: "+str(right/(right+wrong))+" ,正確:"+str(right)+" ,錯誤:"+str(wrong))

Euclidean Distance 正確率: 0.2291220556745182 ,正確:107 ,錯誤:360


## 結果呈現

In [27]:
Testpath = ['.\images\Testdata\M','.\images\Testdata\J','.\images\Testdata\\N','.\images\Testdata\S','.\images\Testdata\Z']
Testfiles = [[],[],[],[],[]]
# r=root, d=directories, f = files
for index in range(5):
    for r, d, f in os.walk(Testpath[index]):
        for file in f:
            Testfiles[index].append(os.path.join(r, file))

In [77]:
import matplotlib as mpl
import random
mpl.rc('font',family ='Noto Sans CJK TC')

In [147]:
from ipywidgets import interact_manual

def NameLabel(people):
    if(people==0):
        return "敏珠"
    elif(people==1):
        return "Jessica"
    elif(people==2):
        return "娜京"
    elif(people==3):
        return "Sinb"
    else:
        return "子瑜"

def ShowPredict():
    TestPeople = random.randint(0,4)
    TestImage = random.randint(0,(len(Testfiles[TestPeople])-1))
    NowTestfile = Testfiles[TestPeople][TestImage]

    plt.figure(figsize=(15, 15))
    img = face_recognition.load_image_file(NowTestfile)
    # (top, right, bottom, left)
    (top, right, bottom, left) = face_recognition.face_locations(img)[0]
    # rectangle(要畫的圖, 左上座標, 右下座標, 顏色, 粗細)
    size = int(img.shape[0] / 100)
    cv2.rectangle(img, (left, top), (right, bottom), (0, 130, 266), size)

    #=========================perdict face
    Testface_encoding = face_recognition.face_encodings(img)
    Testdata_x = np.array(Testface_encoding)
    #https://stackoverflow.com/questions/43017017/keras-model-predict-for-a-single-image
    Testdata_x = Testdata_x.reshape(1,128, 1, 1)#注意這裡最前面要加一個1
    predict = model1.predict_classes(Testdata_x)
    predictLabel = NameLabel(predict[0])
    #===========================

    name = NameLabel(TestPeople)

    plt.title("Label: " + name +"   ,   Predict: "+predictLabel, fontsize=20)
    plt.axis("off")
    plt.imshow(img)


    
interact_manual(ShowPredict);

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

### 結果呈現2